In [ ]:
import jax
import jaxlib

print("jax version:", jax.__version__)
print("jaxlib version:", jaxlib.__version__)

import scvi
import h5py
import anndata as ad
import pandas as pd
import scipy.sparse as sp


In [ ]:
# atse_anndata = []
# with h5py.File("/Users/smriti/documents/Research_Knowles_Lab/scvi-tools-splicing/ann_data/ATSE_Anndata_Object_BRAIN_only_20241105_wLeafletFAPSIs.h5ad", "r") as file:
#     for attr, value in file.attrs.items():
#         print(f"{attr}: {value}")
#     def print_name(name):
#         print(name)
    
#     file.visit(print_name)

# with h5py.File("/Users/smriti/documents/Research_Knowles_Lab/scvi-tools-splicing/ann_data/GE_Anndata_Object_BRAIN_only_20241105.h5ad", "r") as file:
#     for attr, value in file.attrs.items():
#         print(f"{attr}: {value}")


# try:
#     with h5py.File('/Users/smriti/documents/Research_Knowles_Lab/scvi-tools-splicing/ann_data/ATSE_Anndata_Object_BRAIN_only_20241105_wLeafletFAPSIs.h5ad', 'r') as f:
#         for key in f.keys():
#             print(f"Dataset name: {key}")
#             # Try to read individual datasets
#             try:
#                 data = f[key][:]
#                 print(data)
#             except Exception as e:
#                 print(f"Error reading dataset {key}: {e}")
# except Exception as e:
#     print(f"Error opening file: {e}")
    
atse_anndata = ad.read_h5ad('/Users/smriti/documents/Research_Knowles_Lab/multivi_tools_splicing/ann_data/ATSE_Anndata_Object_BRAIN_only_20241105_wLeafletFAPSIs.h5ad')

In [ ]:
atse_anndata.var

In [ ]:
atse_anndata.obs

In [ ]:
ge_anndata = ad.read_h5ad('/Users/smriti/documents/Research_Knowles_Lab/multivi_tools_splicing/ann_data/GE_Anndata_Object_BRAIN_only_20241105.h5ad')

In [ ]:
ge_anndata.var

In [ ]:
ge_anndata.obs

1. Preparing var data. As per the AnnData used in MultiVI, we have ID, modality, and n_cells with the index corresponding
to the exact data point (gene or junction id)

In [ ]:
gene_expr_var = pd.DataFrame({
    'ID': ge_anndata.var['mouse_gene_name'],
    'modality': 'Gene_Expression',
    'n_cells': ge_anndata.var['n_cells']
})
gene_expr_var.index = ge_anndata.var.index

In [ ]:
splicing_var = pd.DataFrame({
    'ID': atse_anndata.var['junction_id'],
    'modality': 'Splicing',
    'n_cells': atse_anndata.var['num_cells_with_junc']
})
splicing_var.index = atse_anndata.var.index  

In [ ]:
combined_var = pd.concat([gene_expr_var, splicing_var])

2. Preparing obs data. 

In [ ]:
combined_obs = pd.DataFrame({
    'batch_id': atse_anndata.obs['batch'],
    'modality': 'paired'
})
combined_obs.index = atse_anndata.obs.index

3. Getting Count Data

In [ ]:
splicing_counts = atse_anndata.layers['junc_ratio']
gene_expr_counts = ge_anndata.layers['raw_counts']
combined_counts = sp.hstack([gene_expr_counts, splicing_counts])

In [ ]:
combined_adata = ad.AnnData(
    X=combined_counts,
    obs=combined_obs,
    var=combined_var
)

In [ ]:
print(combined_adata)

In [ ]:
combined_adata.var

In [ ]:
combined_adata.obs

In [ ]:
combined_adata.write("/Users/smriti/documents/Research_Knowles_Lab/multivi_tools_splicing/ann_data/TMS_BRAINONLY_Combined_GE_ATSE.h5ad")